# **Final project: StackOverflow assistant bot**
In this project, we will combine everything we have learned about Natural Language Processing to construct a dialogue chat bot, which will be able to:

* answer programming-related questions (using 
StackOverflow dataset);
* chit-chat and simulate dialogue on all non programming-related questions.

For a chit-chat mode we will use a pre-trained neural network engine available from [ChatterBot](https://github.com/gunthercox/ChatterBot). Those who aim at honor certificates for our course or are just curious, will train their own models for chit-chat.

**Data description**

To detect intent of users questions we will need two text collections:

* `tagged_posts.tsv` — StackOverflow posts, tagged with one programming language (positive samples).
* `dialogues.tsv` — dialogue phrases from movie subtitles (negative samples).

In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_project()

import sys
sys.path.append("..")
from common.download_utils import download_project_resources

download_project_resources()

--2021-08-09 22:51:08--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1939 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.89K  --.-KB/s    in 0s      

2021-08-09 22:51:08 (17.2 MB/s) - ‘setup_google_colab.py’ saved [1939/1939]

File data/dialogues.tsv is already downloaded.
File data/tagged_posts.tsv is already downloaded.


For those questions, that have programming-related intent, we will proceed as follow predict programming language (only one tag per question allowed here) and rank candidates within the tag using embeddings. For the ranking part, you will need:

* `word_embeddings.tsv `— word embeddings, that you trained with StarSpace in the 3rd assignment. It's not a problem if you didn't do it, because we can offer an alternative solution for you.

As a result of this notebook, you should obtain the following new objects that you will then use in the running bot:

* `intent_recognizer.pkl` — intent recognition model;
* `tag_classifier.pkl` — programming language classification model;
* `tfidf_vectorizer.pkl` — vectorizer used during training;
* `thread_embeddings_by_tags` — folder with thread embeddings, arranged by tags.

Some functions will be reused by this notebook and the scripts, so we put them into utils.py file. Don't forget to open it and fill in the gaps!

In [17]:
!rm utils.py
!rm dialogue_manager.py

In [20]:
from utils import *
from dialogue_manager import DialogueManager

In [21]:
file = open('utils.py')
lines = file. readlines()
for line in lines:
  print(line)

import nltk

import pickle

import re

import numpy as np

import csv



nltk.download('stopwords')

from nltk.corpus import stopwords



# Paths for all resources for the bot.

RESOURCE_PATH = {

    'INTENT_RECOGNIZER': 'intent_recognizer.pkl',

    'TAG_CLASSIFIER': 'tag_classifier.pkl',

    'TFIDF_VECTORIZER': 'tfidf_vectorizer.pkl',

    'THREAD_EMBEDDINGS_FOLDER': 'thread_embeddings_by_tags',

    'WORD_EMBEDDINGS': 'data/word_embeddings.tsv',

}





def text_prepare(text):

    """Performs tokenization and simple preprocessing."""

    

    replace_by_space_re = re.compile('[/(){}\[\]\|@,;]')

    bad_symbols_re = re.compile('[^0-9a-z #+_]')

    stopwords_set = set(stopwords.words('english'))



    text = text.lower()

    text = replace_by_space_re.sub(' ', text)

    text = bad_symbols_re.sub('', text)

    text = ' '.join([x for x in text.split() if x and x not in stopwords_set])



    return text.strip()





def load_embeddings(embeddings_path):

    """Loads pre-trai

In [22]:
file = open('dialogue_manager.py')
lines = file. readlines()
for line in lines:
  print(line)

import os

from sklearn.metrics.pairwise import pairwise_distances_argmin



from chatterbot import ChatBot

from chatterbot.trainers import ChatterBotCorpusTrainer

from utils import *





class ThreadRanker(object):



    def __init__(self, paths):

        self.word_embeddings, self.embeddings_dim = load_embeddings(paths['WORD_EMBEDDINGS'])

        self.thread_embeddings_folder = paths['THREAD_EMBEDDINGS_FOLDER']



    def __load_embeddings_by_tag(self, tag_name):

        embeddings_path = os.path.join(self.thread_embeddings_folder, tag_name + ".pkl")

        thread_ids, thread_embeddings = unpickle_file(embeddings_path)

        return thread_ids, thread_embeddings



    def get_best_thread(self, question, tag_name):

        """ Returns id of the most similar thread for the question.

            The search is performed across the threads with a given tag.

        """

        thread_ids, thread_embeddings = self.__load_embeddings_by_tag(tag_name)



        # HINT: you ha

#**Part I. Intent and language recognition¶**

We want to write a bot, which will not only **answer programming-related questions**, but also will be able to **maintain a dialogue**. We would also like to detect the intent of the user from the question (we could have had a 'Question answering mode' check-box in the bot, but it wouldn't fun at all, would it?). So the first thing we need to do is to **distinguish programming-related questions from general ones**.

It would also be good to predict which programming language a particular question referees to. By doing so, we will speed up question search by a factor of the number of languages (10 here), and exercise our text classification skill a bit. :)

In [2]:
import numpy as np
import pandas as pd
import pickle
import re

from sklearn.feature_extraction.text import TfidfVectorizer

**Data preparation**

In the first assignment (Predict tags on StackOverflow with linear models), you have already learnt how to preprocess texts and do TF-IDF tranformations. Reuse your code here. In addition, you will also need to [dump](https://docs.python.org/3/library/pickle.html#pickle.dump) the TF-IDF vectorizer with pickle to use it later in the running bot.

In [4]:
def tfidf_features(X_train, X_test, vectorizer_path):
    """Performs TF-IDF transformation and dumps the model."""    
    tfidf_vectorizer = TfidfVectorizer(
        ngram_range = (1,2),
        min_df = 5,
        max_df = 0.9,
        token_pattern = '(\S+)'
    )

    # Train a vectorizer on X_train data.
    tfidf_vectorizer.fit(X_train)

    # Transform X_train and X_test data.
    X_train = tfidf_vectorizer.transform(X_train)
    X_test = tfidf_vectorizer.transform(X_test)

    # Pickle the trained vectorizer to 'vectorizer_path'
    # Don't forget to open the file in writing bytes mode.
    vectorizer_file = open(vectorizer_path, 'wb')

    pickle.dump(tfidf_vectorizer, vectorizer_file)
    
    return X_train, X_test

Now, load examples of two classes. Use a subsample of stackoverflow data to balance the classes. You will need the full data later.

In [5]:
sample_size = 200000

dialogue_df = pd.read_csv('data/dialogues.tsv', sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv('data/tagged_posts.tsv', sep='\t').sample(sample_size, random_state=0)


Check how the data look like:

In [6]:
dialogue_df.head()

,text,tag
82925,"Donna, you are a muffin.",dialogue
48774,He was here last night till about two o'clock....,dialogue
55394,"All right, then make an appointment with her s...",dialogue
90806,"Hey, what is this-an interview? We're supposed...",dialogue
107758,Yeah. He's just a friend of mine I was trying ...,dialogue


In [7]:
stackoverflow_df.head()

,post_id,title,tag
2168983,43837842,Efficient Algorithm to compose valid expressio...,python
1084095,15747223,Why does this basic thread program fail with C...,c_cpp
1049020,15189594,Link to scroll to top not working,javascript
200466,3273927,Is it possible to implement ping on windows ph...,c#
1200249,17684551,GLSL normal mapping issue,c_cpp


Apply *text_prepare* function to preprocess the data.

If you filled in the file, but NotImplementedError is still displayed, please refer to [this thread](https://github.com/hse-aml/natural-language-processing/issues/27).

In [8]:
from utils import text_prepare

In [9]:
dialogue_df['text'] = dialogue_df['text'].map(text_prepare)
stackoverflow_df['title'] = stackoverflow_df['title'].map(text_prepare)

# **Intent recognition**

We will do a binary classification on TF-IDF representations of texts. Labels will be either `dialogue` for general questions or `stackoverflow` for programming-related questions. First, prepare the data for this task:

* concatenate `dialogue` and `stackoverflow` examples into one sample
* split it into train and test in proportion 9:1, use *random_state=0* for reproducibility
* transform it into TF-IDF featuresIntent recognition

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

X_train_tfidf, X_test_tfidf = tfidf_features(X_train, X_test,'./tfidf_vectorizer.pkl')

Train size = 360000, test size = 40000


In [12]:
X_train_tfidf.shape

(360000, 67481)

In [13]:
X_test_tfidf.shape

(40000, 67481)

Train the **intent recognizer** using LogisticRegression on the train set with the following parameters: *penalty='l2', C=10, random_state=0*. Print out the accuracy on the test set to check whether everything looks good.

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [15]:
intent_recognizer=LogisticRegression(solver='newton-cg', C=10, penalty='l2', random_state=0)
intent_recognizer.fit(X_train_tfidf, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
X_test_tfidf.shape

(40000, 67481)

In [17]:
# Check test accuracy.
y_test_pred = intent_recognizer.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.991575


Dump the classifier to use it in the running bot.

In [18]:
pickle.dump(intent_recognizer, open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'wb'))

**Programming language classification¶**

We will train one more classifier for the programming-related questions. It will predict exactly one tag (=programming language) and will be also based on Logistic Regression with TF-IDF features.

First, let us prepare the data for this task.

In [19]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 160000, test size = 40000


Let us reuse the TF-IDF vectorizer that we have already created above. It should not make a huge difference which data was used to train it.

In [21]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

X_train_tfidf, X_test_tfidf = vectorizer.transform(X_train), vectorizer.transform(X_test)


Train the **tag classifier** using OneVsRestClassifier wrapper over LogisticRegression. Use the following parameters: *penalty='l2', C=5, random_state=0*.

In [22]:
from sklearn.multiclass import OneVsRestClassifier

In [23]:
tag_classifier = OneVsRestClassifier(LogisticRegression(solver='newton-cg', penalty="l2", C=5, random_state=0))
tag_classifier.fit(X_train_tfidf,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=5, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=0,
                                                 solver='newton-cg', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [24]:
# Check test accuracy.
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.8007



Dump the classifier to use it in the running bot.

In [25]:
pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

# **Part II. Ranking questions with embeddings¶**
To find a relevant answer (a thread from StackOverflow) on a question you will use vector representations to calculate similarity between the question and existing threads. We already had `question_to_vec` function from the assignment 3, which can create such a representation based on word vectors.

However, it would be costly to compute such a representation for all possible answers in *online mode* of the bot (e.g. when bot is running and answering questions from many users). This is the reason why you will create a database with pre-computed representations. These representations will be arranged by non-overlaping tags (programming languages), so that the search of the answer can be performed only within one tag each time. This will make our bot even more efficient and allow not to store all the database in RAM.

Load StarSpace embeddings which were trained on Stack Overflow posts. These embeddings were trained in *supervised mode *for duplicates detection on the same corpus that is used in search. We can account on that these representations will allow us to find closely related answers for a question.

If for some reasons you didn't train StarSpace embeddings in the assignment 3, you can use [pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google. All instructions about how to work with these vectors were provided in the same assignment. However, we highly recommend to use StarSpace's embeddings, because it contains more appropriate embeddings. If you chose to use Google's embeddings, delete the words, which are not in Stackoverflow data.

In [6]:
starspace_embeddings, embeddings_dim = load_embeddings('data/word_embeddings.tsv')

Since we want to precompute representations for all possible answers, we need to load the whole posts dataset, unlike we did for the intent classifier:

In [7]:
posts_df = pd.read_csv('data/tagged_posts.tsv', sep='\t')

Look at the distribution of posts for programming languages (tags) and find the most common ones. You might want to use pandas groupby and count methods:

In [8]:
counts_by_tag = posts_df.groupby("tag")["tag"].count()

Now for each tag you need to create two data structures, which will serve as online search index:

* `tag_post_ids` — a list of post_ids with shape (`counts_by_tag[tag]`,). It will be needed to show the title and link to the thread;
* `tag_vectors` — a matrix with shape (`counts_by_tag[tag]`, `embeddings_dim`) where embeddings for each answer are stored.

Implement the code which will calculate the mentioned structures and dump it to files. It should take several minutes to compute it.

In [9]:
import os
os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)

for tag, count in counts_by_tag.items():
    tag_posts = posts_df[posts_df['tag'] == tag]
    
    tag_post_ids = tag_posts["post_id"].tolist()
    
    tag_vectors = np.zeros((count, embeddings_dim), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i, :] = question_to_vec(question=title, embeddings=starspace_embeddings, dim=embeddings_dim) 

    # Dump post ids and vectors to a file.
    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))

# **Part III. Putting all together¶**

Now let's combine everything that we have done and enable the bot to maintain a dialogue. We will teach the bot to sequentially determine the intent and, depending on the intent, select the best answer. As soon as we do this, we will have the opportunity to chat with the bot and check how well it answers questions.

Implement Dialogue Manager that will generate the best answer. In order to do this, you should open *dialogue_manager.py* and fill in the gaps.

In [1]:
!pip install chatterbot

In [2]:
!pip install chatterbot_corpus

In [3]:
from utils import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from dialogue_manager import DialogueManager

In [5]:
dialogue_manager = DialogueManager(RESOURCE_PATH)

Loading resources...

Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


Now we are ready to test our chat bot! Let's chat with the bot and ask it some questions. Check that the answers are reasonable.

In [6]:
questions = [
    "Hey", 
    "How are you doing?", 
    "What's your hobby?", 
    "How to write a loop in python?",
    "How to delete rows in pandas?",
    "python3 re",
    "What is the difference between c and c++",
    "Multithreading in Java",
    "Catch exceptions C++",
    "What is AI?",
]

for question in questions:
    answer = dialogue_manager.generate_answer(question)
    print('Q: %s\nA: %s \n' % (question, answer))

Q: Hey
A: What a thing to say to another being. 

Q: How are you doing?
A: I am doing well. 

Q: What's your hobby?
A: Playing Soccer, Painting, Writing are my hobbies. How about you? 

Q: How to write a loop in python?
A: I think its about python
This thread might help you: https://stackoverflow.com/questions/337 

Q: How to delete rows in pandas?
A: I think its about python
This thread might help you: https://stackoverflow.com/questions/337 

Q: python3 re
A: I think its about python
This thread might help you: https://stackoverflow.com/questions/337 

Q: What is the difference between c and c++
A: I think its about c_cpp
This thread might help you: https://stackoverflow.com/questions/263 

Q: Multithreading in Java
A: I think its about java
This thread might help you: https://stackoverflow.com/questions/564 

Q: Catch exceptions C++
A: I think its about c_cpp
This thread might help you: https://stackoverflow.com/questions/263 

Q: What is AI?
A: I think its about java
This thread mi

In [10]:
!zip -r /content/content.zip /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2021.07.16/ (stored 0%)
  adding: content/.config/logs/2021.07.16/13.19.42.311157.log (deflated 54%)
  adding: content/.config/logs/2021.07.16/13.20.21.584309.log (deflated 55%)
  adding: content/.config/logs/2021.07.16/13.20.22.328313.log (deflated 54%)
  adding: content/.config/logs/2021.07.16/13.20.04.172291.log (deflated 54%)
  adding: content/.config/logs/2021.07.16/13.19.23.269640.log (deflated 91%)
  adding: content/.config/logs/2021.07.16/13.19.58.237112.log (deflated 86%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_u